Tokenisation

**Instructor: Haiqin Yang**: yanghaiqin@sztu.edu.cn

---
# Setup

In [ ]:
# @title Imports
!pip install tokenizers
from urllib.request import urlopen
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
nltk.download('punkt')
nltk.download('stopwords')
!pip install SentencePiece
from transformers import MT5Tokenizer, GPT2LMHeadModel, TextGenerationPipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00


---
# Read text


## From webpage

In [ ]:
# Download comments from webpage
html = urlopen('https://www.hplovecraft.com/writings/texts/fiction/cc.aspx').read()

In [ ]:
html

In [ ]:
# Remove html markup
raw = BeautifulSoup(html).get_text()

In [ ]:
raw

'\n\n"The Call of Cthulhu" by H. P. Lovecraft\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\nHome\n\xa0\xa0\xa0\nHis Life\n\xa0\xa0\xa0\nHis Writings\n\xa0\xa0\xa0\nHis Creations\n\xa0\xa0\xa0\nHis Study\n\xa0\xa0\xa0\nPopular Culture\n\xa0\xa0\xa0\nInternet Resources\n\xa0\xa0\xa0\nAbout This Site\n\xa0\n\n\n\n\n\n\n\n\nThe Call of CthulhuBy H. P. Lovecraft\n\n\n\n(Found Among the Papers of the LateFrancis Wayland Thurston, of Boston)\n“Of such great powers or beings there may be conceivably a survival\xa0.\xa0.\xa0.\r\na survival of a hugely remote period when\xa0.\xa0.\xa0. consciousness was manifested,\r\nperhaps, in shapes and forms long since withdrawn before the tide of advancing humanity\xa0.\xa0.\xa0.\r\nforms of which poetry and legend alone have caught a flying memory and called them gods, monsters,\r\nmythical beings of all sorts and kinds.\xa0.\xa0.\xa0.”\n—Algernon Blackwood.\nI.The Horror in Clay.\r\n\r\nThe most merciful thing in the world, I think, is the inability of the 

In [ ]:
# trim the text to where you want it to begin
ind_start = re.search('“Of such great powers', raw).start()
raw = raw[ind_start:]

In [ ]:
ind_start

316

In [ ]:
raw

'“Of such great powers or beings there may be conceivably a survival\xa0.\xa0.\xa0.\r\na survival of a hugely remote period when\xa0.\xa0.\xa0. consciousness was manifested,\r\nperhaps, in shapes and forms long since withdrawn before the tide of advancing humanity\xa0.\xa0.\xa0.\r\nforms of which poetry and legend alone have caught a flying memory and called them gods, monsters,\r\nmythical beings of all sorts and kinds.\xa0.\xa0.\xa0.”\n—Algernon Blackwood.\nI.The Horror in Clay.\r\n\r\nThe most merciful thing in the world, I think, is the inability of the human mind to correlate\r\nall its contents. We live on a placid island of ignorance in the midst of black seas of infinity,\r\nand it was not meant that we should voyage far. The sciences, each straining in its own direction,\r\nhave hitherto harmed us little; but some day the piecing together of dissociated knowledge will\r\nopen up such terrifying vistas of reality, and of our frightful position therein, that we shall\r\neither g

In [ ]:
# save contents to text file
with open ('cthulhu.txt', 'w') as f:
  f.write(raw)

## From text file

In [ ]:
# read the file we just saved
with open ('cthulhu.txt', 'r') as f:
  raw = f.read()

In [ ]:
raw

'“Of such great powers or beings there may be conceivably a survival\xa0.\xa0.\xa0.\na survival of a hugely remote period when\xa0.\xa0.\xa0. consciousness was manifested,\nperhaps, in shapes and forms long since withdrawn before the tide of advancing humanity\xa0.\xa0.\xa0.\nforms of which poetry and legend alone have caught a flying memory and called them gods, monsters,\nmythical beings of all sorts and kinds.\xa0.\xa0.\xa0.”\n—Algernon Blackwood.\nI.The Horror in Clay.\n\nThe most merciful thing in the world, I think, is the inability of the human mind to correlate\nall its contents. We live on a placid island of ignorance in the midst of black seas of infinity,\nand it was not meant that we should voyage far. The sciences, each straining in its own direction,\nhave hitherto harmed us little; but some day the piecing together of dissociated knowledge will\nopen up such terrifying vistas of reality, and of our frightful position therein, that we shall\neither go mad from the revelat

---
# Tokenization

## Using whitespace

In [ ]:
# segment words using whitespace
tokens = raw.split() # Python's built-in function, default seperaotr is whitespace; but you can change it to other seperators
sent_tokens = raw.split('.')

In [ ]:
tokens

In [ ]:
sent_tokens

## Using NLTK's tokenizer

In [ ]:
tokens = word_tokenize(raw)

In [ ]:
tokens

In [ ]:
# Show frequncy of each word
token_freq = FreqDist(tokens) #nltk's function to find how many times each token occurs in the text

In [ ]:
token_freq.most_common(50)

# Text normalization

### Remove punctutation

In [ ]:
tokens = [t for t in tokens if t.isalpha()]

In [ ]:
tokens

In [ ]:
token_freq = FreqDist(tokens)
token_freq.most_common(50)

### Case folding

In [ ]:
tokens_lower = [t.lower() for t in tokens]

In [ ]:
tokens_lower

### Lemmatization

#### Using Porter stemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
ps.stem('computational')

'comput'

In [ ]:
ps.stem('linguistics')

'linguist'

In [ ]:
tokens_stemmed = [ps.stem(w) for w in tokens_lower]

In [ ]:
tokens_stemmed

In [ ]:
# save normalized tokens
with open('tokens_normed.txt', 'w') as f:
  f.write('\n'.join(tokens_lower))

#### Using BPE tokenizer

In [ ]:
# import BPE tokenizer and train on the normalized text
bpe_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
bpe_tokenizer.train(['tokens_normed.txt'], trainer)

In [ ]:
# test BPE tokenizer on unseen words
output = bpe_tokenizer.encode('computational linguistics')
output.tokens

['comp', 'ut', 'at', 'ion', 'al', '[UNK]', 'ling', 'u', 'ist', 'ic', 's']

#### Using mGPT tokenizer

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("THUMT/mGPT")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'T5Tokenizer'.


In [ ]:
input_ids = tokenizer('computational linguistics').input_ids

In [ ]:
input_ids

[93928, 65662, 259, 151025, 263, 1]

In [ ]:
[tokenizer.decode(i) for i in input_ids]

['comput', 'ational', '', 'linguistic', 's', '</s>']

In [ ]:
text = '当我还只有六岁的时候在一本描写原始森林的名叫真实的故事的书中看到了一幅精彩的插画'

In [ ]:
input_ids = tokenizer(text).input_ids

In [ ]:
input_ids

In [ ]:
[tokenizer.decode(i) for i in input_ids]

In [ ]:
with open ('tokens_normed.txt', 'r') as f:
  tokens_normed = f.read()

In [ ]:
tokenizer(text).input_ids